In [ ]:
import pandas as pd
from IPython.display import display
df = pd.read_excel('/kaggle/input/dataset/dataset.xlsx' , engine='openpyxl')

# Combining columns
df["Question"] = ("Description: " + df.Description.str.strip() + "; Patient: " + df.Patient.str.strip())

# Droping Description and Patient columns
df.drop(['Description', 'Patient'], axis=1, inplace=True)

df.rename(columns={'Doctor': 'Answer'}, inplace=True)

df = df[['Question', 'Answer']]
# Displaying the first 5 entries of the DataFrame in columns
display(df.head())

,Question,Answer
0,Description: Q. What does abutment of the nerv...,Hi. I have gone through your query with dilige...
1,Description: Q. What should I do to reduce my ...,Hi. You have really done well with the hypothy...
2,Description: Q. I have started to get lots of ...,Hi there Acne has multifactorial etiology. Onl...
3,Description: Q. Why do I have uncomfortable fe...,Hello. The popping and discomfort what you fel...
4,Description: Q. My symptoms after intercourse ...,Hello. The HIV test uses a finger prick blood ...


# Preprocessing

In [ ]:

import re

def decontractions(phrase):
    """decontracted takes text and convert contractions into natural form.
     ref: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python/47091490#47091490"""
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"won\’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’s", " is", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)

    return phrase

def preprocess(text):
    if isinstance(text, str):

    # Converting all the text into lowercase
    # Removing the words between brackets ()
    # Removing these characters: {'$', ')', '?', '"', '’', '.',  '°', '!', ';', '/', "'", '€', '%', ':', ',', '('}
    # Replacing these special characters with space: '\u200b', '\xa0', '-', '/'

        text = text.lower()
        text = decontractions(text)
        text = re.sub(r'\(.*?\)', '', text)  # Remove text between brackets
        text = re.sub('[$)\?"’.°!;\'€%:,(/]', '', text)
        text = re.sub('\u200b', ' ', text)
        text = re.sub('\xa0', ' ', text)
        text = re.sub('-', ' ', text)
        return text

# Function to preprocess and clean the DataFrame
def preprocess_dataframe(df):
    # Applying preprocessing to the 'Question' column
    df['Question'] = df['Question'].apply(preprocess)

    # Applying preprocessing to the 'Answer' column
    df['Answer'] = df['Answer'].apply(preprocess)

    # Removing null values
    df.dropna(inplace=True)

    # Removing duplicates
    df.drop_duplicates(inplace=True)

    return df


df = preprocess_dataframe(df)

# Display the first 5 entries of the DataFrame
display(df.head())


,Question,Answer
0,description q what does abutment of the nerve ...,hi i have gone through your query with diligen...
1,description q what should i do to reduce my we...,hi you have really done well with the hypothyr...
2,description q i have started to get lots of ac...,hi there acne has multifactorial etiology only...
3,description q why do i have uncomfortable feel...,hello the popping and discomfort what you felt...
4,description q my symptoms after intercourse th...,hello the hiv test uses a finger prick blood s...


In [ ]:
from sklearn.model_selection import train_test_split

# Splitting the data into features (X) and labels (y)
X = df['Question']  # Assuming 'Question' column contains features
y = df['Answer']    # Assuming 'Answer' column contains labels

# Splitting the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Displaying the shape of the training and testing sets
print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)


Training set shape: (197228,) (197228,)
Testing set shape: (49308,) (49308,)


In [ ]:
pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 38.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 13.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from IPython.display import display, FileLink
from io import BytesIO
from sklearn.model_selection import train_test_split

def create_pdf_from_dataframe(df, pdf_file):
    # Create a BytesIO buffer to hold the PDF data
    pdf_buffer = BytesIO()

    # Creating a PDF document
    c = canvas.Canvas(pdf_buffer, pagesize=letter)
    width, height = letter

    # Setting initial y-coordinate for content
    y = height - 50

    # Looping through each row in the DataFrame and add content to PDF
    for index, row in df.iterrows():
        question = row['Question']
        answer = row['Answer']

        # Writing question and answer to PDF
        c.drawString(50, y, "Question: {}".format(question))
        y -= 20
        c.drawString(50, y, "Answer: {}".format(answer))
        y -= 30  # Adjust spacing between questions and answers


        if y < 50:
            c.showPage()
            y = height - 50

    # Saving
    c.save()


    pdf_buffer.seek(0)


    with open(pdf_file, "wb") as f:
        f.write(pdf_buffer.read())


    display(FileLink(pdf_file))

# Split the dataframe into training and testing sets
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Save PDF files for training and testing sets
create_pdf_from_dataframe(df_train, "Training_Dataset.pdf")
create_pdf_from_dataframe(df_test, "Testing_Dataset.pdf")


/kaggle/working/Training_Dataset.pdf

/kaggle/working/Testing_Dataset.pdf

In [ ]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 56.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is 

In [ ]:
pip install PyDrive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.4/987.4 kB 23.5 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for PyDrive: filename=PyDrive-1.3.1-py3-none-any.whl size=27434 sha256=e10aff19a34c42d58c330d0fddc27a01bccc89a282bd3d46ab6120a1a0802908
  Stored in directory: /root/.cache/pip/wheels/63/79/df/924c22c080c9dac1a57f611baa837fe0bc3daec1500b27f23b
Successfully built PyDrive
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 19.4 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


# Embeddings

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Specify the individual PDF file path
PDF_PATH_COLAB = '/kaggle/input/training-dataset/Training_Dataset.pdf'  # Adjust path accordingly

# Specify FAISS database path in Google Drive
DB_FAISS_PATH_DRIVE = '/kaggle/working/db_faiss'  # Adjust path accordingly

# Create vector database
def create_vector_db():
    # Load a single PDF file directly
    loader = PyPDFLoader(PDF_PATH_COLAB)
    documents = loader.load()

    # Split the loaded document into smaller chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)

    # Create embeddings using a pre-trained model
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', model_kwargs={'device': 'cuda'})
    db = FAISS.from_documents(texts, embeddings)

    # Save FAISS database
    db.save_local(DB_FAISS_PATH_DRIVE)

if __name__ == "__main__":
    create_vector_db()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
pip install ctransformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 37.7 MB/s eta 0:00:0000:01:00:01
Note: you may need to restart the kernel to use updated packages.


# RAG implementation

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_community.llms import CTransformers
from langchain.chains import RetrievalQA
from IPython.display import display, FileLink

# Specifying the paths
PDF_PATH_COLAB = '/kaggle/input/training-dataset/Training_Dataset.pdf'


OUTPUT_DIR = '/kaggle/working/'

# Specifying the filename for the FAISS database
DB_FAISS_FILENAME = 'db_faiss.faiss'  # Adjust filename accordingly

# Specifying the filename for the LLM model
LLM_MODEL_FILENAME = 'llm_model'  # Adjust filename accordingly

# Custom prompt template
custom_prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know.

Context: {context}
Question: {question}

Only return the helpful answer below.
Helpful answer:
"""

def set_custom_prompt():
    prompt = PromptTemplate(template=custom_prompt_template, input_variables=['context', 'question'])
    return prompt

def load_llm():
    llm = CTransformers(
        model="TheBloke/Llama-2-7B-Chat-GGML",
        model_type="llama",
        max_new_tokens=512,
        temperature=0.5
    )
    return llm

def create_vector_db():
    # Load a single PDF file directly
    loader = PyPDFLoader(PDF_PATH_COLAB)
    documents = loader.load()

    # Splitting
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)

    # Creating embeddings using a pre-trained model
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', model_kwargs={'device': 'cuda'})
    db = FAISS.from_documents(texts, embeddings)

    # Saving FAISS database
    faiss_db_path = OUTPUT_DIR + DB_FAISS_FILENAME
    db.save_local(faiss_db_path)

    return faiss_db_path

def retrieval_qa_chain(llm, prompt, db):
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type='stuff',
        retriever=db.as_retriever(search_kwargs={'k': 2}),
        return_source_documents=True,
        chain_type_kwargs={'prompt': prompt}
    )
    return qa_chain

def qa_bot():
    # Load the LLM model
    llm = load_llm()


    faiss_db_path = create_vector_db()
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', model_kwargs={'device': 'cuda'})
    db = FAISS.load_local(faiss_db_path, embeddings, allow_dangerous_deserialization=True)

    qa_prompt = set_custom_prompt()

    # Creating the QA chain
    qa = retrieval_qa_chain(llm, qa_prompt, db)

    return qa


def final_result(query):
    qa_result = qa_bot()
    response = qa_result({'query': query})
    return response

if __name__ == "__main__":
    query = "What is the recommended treatment for hypertension?"
    result = final_result(query)
    print(result)


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

{'query': 'What is the recommended treatment for hypertension?', 'result': "I don't know of any natural ways to treat hypertension. The best approach to treating hypertension is through a combination of lifestyle changes and medication. A low-salt diet, regular exercise, stress reduction techniques, and weight loss can all help to lower blood pressure. Medications such as telmisartan and hydrochlorothiazide are often prescribed to treat high blood pressure.\n\nPlease let me know if there is anything else I can assist you with.", 'source_documents': [Document(page_content='Answer: there are very good treatment available for hypertension first one should consume low salt diet i usually start the patient on telmisartan hydrochlorothiazide combination after ruling out contraindications and titrate for response we can add amlodipine or other drugs if needed', metadata={'source': '/kaggle/input/training-dataset/Training_Dataset.pdf', 'page': 3729}), Document(page_content='Question: descripti

# Evaluation

In [ ]:
import pandas as pd
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import random

# Individual CSV file path
CSV_PATH = '/kaggle/input/training-dataset/Testing_Dataset.pdf'  # Adjust the path to your CSV file

# FAISS database path
DB_FAISS_PATH_DRIVE = '/kaggle/working/db_faiss'  # Adjust path accordingly

# Columns in the CSV file that contain the question and answer data
QUESTION_COLUMN = 'Question'  # Adjust the column name accordingly
ANSWER_COLUMN = 'Answer'  # Adjust the column name accordingly

# Creating vector database
def create_vector_db():
    # Load the CSV file
    df = pd.read_csv(CSV_PATH)

    # Randomly sample two rows
    sampled_df = df.sample(n=2, random_state=1)

    # Extracting the question and answer data from the specified columns
    questions = sampled_df[QUESTION_COLUMN].dropna().tolist()  # Drop any NaN values and convert to a list
    answers = sampled_df[ANSWER_COLUMN].dropna().tolist()  # Drop any NaN values and convert to a list

    # Splitting the question and answer data into smaller chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

    question_documents = []
    for question in questions:
        splitted_texts = text_splitter.split_text(question)
        for split_text in splitted_texts:
            question_documents.append(Document(page_content=split_text))

    answer_documents = []
    for answer in answers:
        splitted_texts = text_splitter.split_text(answer)
        for split_text in splitted_texts:
            answer_documents.append(Document(page_content=split_text))


    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', model_kwargs={'device': 'cpu'})
    question_db = FAISS.from_documents(question_documents, embeddings)
    answer_db = FAISS.from_documents(answer_documents, embeddings)

    # Save FAISS database
    question_db.save_local(DB_FAISS_PATH_DRIVE + '_questions')
    answer_db.save_local(DB_FAISS_PATH_DRIVE + '_answers')

if __name__ == "__main__":
    create_vector_db()


In [ ]:
import pandas as pd
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain_community.llms import CTransformers
from langchain.chains import RetrievalQA
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Paths and filenames
DB_FAISS_QUESTIONS_PATH = '/kaggle/working/db_faiss_questions'
DB_FAISS_ANSWERS_PATH = '/kaggle/working/db_faiss_answers'

# Custom prompt template
custom_prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know.

Context: {context}
Question: {question}

Only return the helpful answer below.
Helpful answer:
"""

def set_custom_prompt():
    prompt = PromptTemplate(template=custom_prompt_template, input_variables=['context', 'question'])
    return prompt

def load_llm():
    llm = CTransformers(
        model="TheBloke/Llama-2-7B-Chat-GGML",
        model_type="llama",
        max_new_tokens=512,
        temperature=0.5
    )
    return llm

def retrieval_qa_chain(llm, prompt, question_db, answer_db):
    # Use the question_db to retrieve relevant question embeddings
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type='stuff',
        retriever=question_db.as_retriever(search_kwargs={'k': 2}),
        return_source_documents=True,
        chain_type_kwargs={'prompt': prompt}
    )
    return qa_chain

def qa_bot():
    # Load the LLM model
    llm = load_llm()

    # Load the FAISS databases for questions and answers
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', model_kwargs={'device': 'cuda'})
    question_db = FAISS.load_local(DB_FAISS_QUESTIONS_PATH, embeddings, allow_dangerous_deserialization=True)
    answer_db = FAISS.load_local(DB_FAISS_ANSWERS_PATH, embeddings, allow_dangerous_deserialization=True)

    qa_prompt = set_custom_prompt()

    # Create the QA chain
    qa = retrieval_qa_chain(llm, qa_prompt, question_db, answer_db)

    return qa, question_db, answer_db

def compute_cosine_similarity(generated_answer, reference_answer):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

    # Encode generated and reference answers
    generated_embedding = model.encode(generated_answer)
    reference_embedding = model.encode(reference_answer)

    # Compute cosine similarity between embeddings
    similarity_score = cosine_similarity([generated_embedding], [reference_embedding])[0][0]

    return similarity_score

def evaluate_rag(queries, expected_answers, threshold=0.7):
    generated_answers = []
    qa, question_db, answer_db = qa_bot()

    # Encode expected answers
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    expected_answer_embeddings = model.encode(expected_answers)

    for query in queries:
        response = qa({'query': query})
        generated_answers.append(response['result'])

    # Encode generated answers
    generated_answer_embeddings = model.encode(generated_answers)

    # Compute cosine similarity between question-answer pairs
    similarity_scores = cosine_similarity(generated_answer_embeddings, expected_answer_embeddings)

    # Convert similarity scores to binary results based on a threshold
    binary_results = similarity_scores.diagonal() >= threshold

    accuracy = binary_results.mean()
    precision = binary_results.sum() / len(binary_results)
    recall = binary_results.sum() / len(binary_results)


    # Log the results for inspection
    for i, (query, expected, generated, score) in enumerate(zip(queries, expected_answers, generated_answers, similarity_scores.diagonal())):
        print(f"Query {i+1}: {query}")
        print(f"Expected Answer: {expected}")
        print(f"Generated Answer: {generated}")
        print(f"Similarity Score: {score}")
        print()

    return accuracy, precision, recall, f1, similarity_scores

if __name__ == "__main__":
    # Load your test data from CSV
    CSV_PATH = '/content/drive/MyDrive/Testing_Dataset.csv'
    test_data = pd.read_csv(CSV_PATH)

    # Sampling the dataframe
    test_data = test_data.sample(n=2, random_state=42)


    queries = test_data['Question'].tolist()
    expected_answers = test_data['Answer'].tolist()

    # Evaluating the RAG method with the threshold value of 0.7
    accuracy, precision, recall, f1, similarity_scores = evaluate_rag(queries, expected_answers, threshold=0.7)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")



# Interface

In [ ]:
import gradio as gr

# Define your previous function for processing queries
def final_result(query):
    # Your previous code for processing the query and generating a response
    response = "This is a placeholder response for the query: '{}'".format(query)
    return response

# Define the Gradio interface function
def interface(query):
    # Call the previous function to get the response
    response = final_result(query)
    return response

# Create the Gradio interface
iface = gr.Interface(fn=interface, inputs="text", outputs="text", title="Health-Bot", description="Ask any medical related question")
iface.launch()

In [ ]:
pip install gradio

ERROR: Could not find a version that satisfies the requirement gradio (from versions: none)
ERROR: No matching distribution found for gradio
Note: you may need to restart the kernel to use updated packages.
